In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True )

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
X = tf.placeholder(tf.float32, [None, 784])
# x_image = (NumOfImage, 28 * 28 * 1)
x_image = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [3]:
# Conv_layer1
# input image = NumOfImage * ( 28 * 28 * 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01)) # size of filter
# ( NumOfImage, 28 * 28 * 32) => filtering result (32 = 필터의 개수)
L1 = tf.nn.conv2d(x_image, W1, strides =[1,1,1,1], padding = 'SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1],
                   strides = [1, 2, 2, 1], padding = 'SAME')
# NumOfImage, 14, 14 , 32 => pooling result 

In [4]:
# Conv_Layer2
# tf.random_normal([row, col, channel, num of filter]) => filter 정의
# row, col은 필터의 크기, channel은 input data의 channel과 같아야함
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64]))
# Conv result =  NumOfImage, 14 * 14 * 64
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding ='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides = [1, 2, 2, 1], padding = 'SAME')
# Pooling result = NumOfImage, 7 * 7 * 64
L2 = tf.reshape(L2, [-1, 7 * 7 * 64]) # NumOfImage, 평면으로
# Fully Cennected 에 넣기 위해서 펼치는 작업


In [10]:
# FC Layer

W3 = tf.get_variable("W2", shape=[7 * 7 * 64, 10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [12]:
# Learning Model
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y : batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.3f}'.format(avg_cost))
    
print('Learning Finished!')

Epoch: 0001 cost = 0
Epoch: 0002 cost = 0
Epoch: 0003 cost = 0
Epoch: 0004 cost = 0
Epoch: 0005 cost = 0
Epoch: 0006 cost = 0
Epoch: 0007 cost = 0
Epoch: 0008 cost = 0
Epoch: 0009 cost = 0
Epoch: 0010 cost = 0
Epoch: 0011 cost = 0
Epoch: 0012 cost = 0
Epoch: 0013 cost = 0
Epoch: 0014 cost = 0
Epoch: 0015 cost = 0
Learning Finished!


In [16]:
# Test Model
prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels}))

r = np.random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

Accuracy: 0.9868
Label:  [7]
Prediction:  [7]
